### DeepAR Model - Bike Rental Training

Note: This data is not a true timeseries as there are lots of gaps

We have data only for first 20 days of each month and model needs to predict the rental for the remaining days of the month. The dataset consists of two years data. DeepAR will shine with true multiple-timeseries dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import datetime

import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
### Import s3 bucket name as environment variable

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [3]:
# set differnt job names when building different models based on choices
# Also have jobnames easily differiantiate according to choice

with_categories = True
if with_categories:
    base_job_name = 'deepar-biketrain-with-categories'
else:
    base_job_name = 'deepar-biketrain-no-categories'

In [4]:
# specify your bucket name and dataset path in that

bucket = os.environ['BUCKET_NAME']
prefix = 'deepar/bikerental'

# This structure allows multiple training and test files for model development and testing

if with_categories:
    s3_data_path = "{}/{}/data_with_categories".format(bucket, prefix)
else:
    s3_data_path = "{}/{}/data".format(bucket,prefix)
    
s3_output_path = "{}/{}/output".format(bucket,prefix)


In [5]:
#s3_data_path, s3_output_path

In [6]:
# function that uploads files to s3 bucket

def write_to_s3(filename, bucket, key):
    with open(filename, 'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [7]:
# upload one or more training files and test files to s3

if with_categories:
    write_to_s3('train_with_categories.json', bucket, 'deepar/bikerental/data_with_categories/train/train_with_categories.json')
    write_to_s3('test_with_categories.json', bucket, 'deepar/bikerental/data_with_categories/test/test_with_categories.json')
else:
    write_to_s3('train.json',bucket, 'deepar/bikerental/data/train/train.json')
    write_to_s3('test.json', bucket, 'deepar/bikerental/data/test/test.json')

In [10]:
# Use spont instance 

use_spot_instances = True
max_run = 3600
max_wait = 3600 if use_spot_instances else None 

job_name = base_job_name

checkpoint_s3_uri = None

if use_spot_instances:
    checkpoint_s3_uri = f's3://{bucket}/{prefix}/checkpoints/{job_name}'
    
#print(f'Checkpoint uri: {checkpoint_s3_uri}')

In [12]:
# Establish a session with AWS

sess = sagemaker.Session()
role = get_execution_role()

In [16]:
# This role contains the permissions needed to train, deploy models
# Sagemaker serveis is trusted to assume this role

#print(role)

In [13]:
# SDK 2 uses image_uris.retrie to get container image

container = sagemaker.image_uris.retrieve("forecasting-deepar",sess.boto_region_name)

print(f'Using DeepAR container {container}')

Using DeepAR container 522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:1


In [14]:
freq = "H" # Data consists hourly data

prediction_length = 288 # need to predict 12 days of data in hours

context_length = 288 # AWS recommends that context lenght to be same as prediction length to look past the same length as pred


In [15]:
# Configure training job

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count = 1,
    instance_type = 'ml.m5.xlarge',
    output_path = 's3://'+s3_output_path,
    sagemaker_session=sess,
    base_job_name= job_name,
    use_spot_instances = use_spot_instances,
    max_run= max_run,
    max_wait = max_wait,
    checkpoint_s3_uri = checkpoint_s3_uri
)

In [13]:
freq, context_length, prediction_length

('H', 288, 288)

In [16]:
# deepar hyperparameters

hyperparameters = {
    "time_freq" : freq,
    "epochs":"400",
    "early_stopping_patience":"10",
    "mini_batch_size": "64",
    "learning_rate" : "5E-4",
    "context_length" : str(context_length),
    "prediction_length" : str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [17]:
hyperparameters

{'time_freq': 'H',
 'epochs': '400',
 'early_stopping_patience': '10',
 'mini_batch_size': '64',
 'learning_rate': '5E-4',
 'context_length': '288',
 'prediction_length': '288',
 'cardinality': 'auto'}

In [18]:
estimator.set_hyperparameters(**hyperparameters)

In [20]:
# creating data channels of train and test files

data_channels = {
    "train" : "s3://{}/train/".format(s3_data_path),
    "test" : "s3://{}/test".format(s3_data_path)
}

In [32]:
#data_channels

In [ ]:
# fitting the model

estimator.fit(inputs=data_channels)

INFO:sagemaker:Creating training-job with name: deepar-biketrain-with-categories-2024-06-06-20-51-20-892


2024-06-06 20:51:21 Starting - Starting the training job...
2024-06-06 20:51:35 Starting - Preparing the instances for training...
2024-06-06 20:52:08 Downloading - Downloading input data.

In [47]:
job_name = estimator.latest_training_job.name

In [21]:
# Hard cde name for now as we stopped the notebook instance
job_name = 'deepar-biketrain-with-categories-2024-06-06-20-51-20-892'

#### Create endpoint using jobname

In [22]:
print('job name: {0}'.format(job_name))

job name: deepar-biketrain-with-categories-2024-06-06-20-51-20-892


In [23]:
# create an endpoint for real-time predictions

endpoint_name = sess.endpoint_from_job(
    job_name = job_name,
    initial_instance_count = 1,
    instance_type = 'ml.m5.large' if use_spot_instances else 'ml.m5.xlarge',
    image_uri = container, 
    role = role
)

--------!